# Physical Environment Simulation

This script simulates the **heat equation** in a simplified rack geometry using the **Gridap.jl** library and the **finite element method (FEM)**.

## 1. The Heat Equation

The heat equation models how temperature evolves in a medium over time due to conduction. The standard form in three dimensions is:

$
\frac{\partial u}{\partial t} - \nabla \cdot (\alpha \nabla u) = f \quad \text{in } \Omega \times (0, T)
$
 where $ u(\mathbf{x}, t) $ istemperature at point $\mathbf{x}$ and time $t$, $ \alpha $ is the thermal diffusivity (may vary in time or space), $ f $ is the internal heat source , $ \Omega $ is the spatial domain  and $ T $ is the  final time.

## 2. Boundary and Initial Conditions

To solve the heat equation, we specify:

- **Initial condition**: This sets the temperature distribution at the beginning of the simulation

    $
      u(\mathbf{x}, 0) = u_0(\mathbf{x}) \quad \text{in } \Omega
    $
  
- **Boundary conditions** : that define how the system interacts with its surroundings along the boundary $ \partial\Omega $ Common types include:

    -  Dirichlet Boundary Condition: this specifies a **fixed temperature** on part of the boundary
        $
    u(\mathbf{x}, t) = g_D(\mathbf{x}, t) \quad \text{on } \Gamma_D \subset \partial\Omega
        $
    - Neumann Boundary Condition : specifies a **heat flux** across the boundary:

        $
    -\alpha \frac{\partial u}{\partial n} = q_N(\mathbf{x}, t) \quad \text{on } \Gamma_N \subset \partial\Omega
        $

    Where $ \partial u / \partial n $ is the derivative in the direction normal to the boundary and $ q_N $ is the         prescribed heat flux (positive means entering the domain).

    This condition arises naturally in the weak form (as surface integrals) and is often used to model heat                 sources/sinks and insulated boundaries (if $ q_N = 0 $)

  -  Robin Boundary Condition (Convective Exchange) this models **convection** between the body and a surrounding            medium (like air):

    $
    -\alpha \frac{\partial u}{\partial n} = h(u - T_{\infty}) \quad \text{on } \Gamma_{\text{ext}} \subset \partial\Omega
    $

    Where $ h $ is the convection heat transfer coefficient and $ T_{\infty} $ is the ambient temperature (air, fluid,      etc.).
     This combines a Neumann-like flux with a temperature difference — commonly used for heat exchange with the              environment.

  
## 3. . Weak Formulation for FEM

To apply the Finite Element Method (FEM), we multiply the heat equation by a test function \( v \) and integrate over the domain \( \Omega \). This gives the **weak (variational) formulation**:

Find $ u(t) \in V $ such that for all $ v \in V $:

$
\int_{\Omega} \frac{\partial u}{\partial t} \, v \, d\Omega + \int_{\Omega} \alpha(t) \nabla u \cdot \nabla v \, d\Omega = \int_{\Omega} f \, v \, d\Omega
$

Where:
- $ V $ is a suitable function space (e.g., $ H^1(\Omega) $) that satisfies the boundary conditions.
- The first term accounts for the **transient (time-dependent)** behavior.
- The second term represents **diffusion** (heat conduction).
- The right-hand side can include **internal heat sources** or be extended to include boundary integrals depending on the type of boundary conditions applied.

This weak form is the basis for constructing the finite element discretization.

In [16]:
using Gridap
using Gridap.FESpaces
using GridapGmsh
using Gridap.CellData

In this case, the computational domains represent simplified rack structures, modeled as homogeneous (single-material) boxes. The heat sources, ie the resistors, are modeled as regions excluded from the domain. These regions impose a prescribed heat flux across the adjacent boundaries of the computational domain. 

The mesh rack_box_placa.msh includes a single heat source represented as a thin plate that spans nearly the entire rack, simulating a planar array of resistive elements. In contrast, the mesh rack2_box_2.msh contains two thinner heat-emitting plates. These configurations allow for the evaluation of different resistor geometries and their impact on the resulting temperature distribution in a straightforward yet effective manner.

The position of the plates reflects the actual location of the resistive components in the physical rack. Additionally, the dimensions of the computational domain match the real-world dimensions of the rack.



In [29]:
model = GmshDiscreteModel("rack_box_placa.msh")
# rack2_box_2.msh 

Info    : Reading 'rack_box_placa.msh'...
Info    : 53 entities
Info    : 2672 nodes
Info    : 14432 elements
Info    : Done reading 'rack_box_placa.msh'


UnstructuredDiscreteModel()

In [18]:
Ω  = Triangulation(model)
dΩ = Measure(Ω,2)

GenericMeasure()

This notebook applies Dirichlet boundary conditions on the external faces of the domain, representing a fixed temperature corresponding to the ambient laboratory temperature. In the vicinity of the heat sources, Neumann boundary conditions are imposed to model the injection of thermal flux into the system.

In [19]:
reffe = ReferenceFE(lagrangian,Float64,1)
V = TestFESpace(model,reffe,dirichlet_tags="ext")

UnconstrainedFESpace()

In [20]:
Γ_source_up = BoundaryTriangulation(model,tags="fuente_up")
dΓ_souce_up= Measure(Γ_source_up,2)

Γ_source_down = BoundaryTriangulation(model,tags="fuente_down")
dΓ_souce_down= Measure(Γ_source_down,2)

Γ_source_y0 = BoundaryTriangulation(model,tags="fuente_y0")
dΓ_souce_y0= Measure(Γ_source_y0,2)

Γ_source_yL = BoundaryTriangulation(model,tags="fuente_yL")
dΓ_souce_yL= Measure(Γ_source_yL,2)

Γ_source_x0 = BoundaryTriangulation(model,tags="fuente_x0")
dΓ_souce_x0= Measure(Γ_source_x0,2)

Γ_source_xL = BoundaryTriangulation(model,tags="fuente_xL")
dΓ_souce_xL= Measure(Γ_source_xL,2)
#n_source = get_normal_vector(Γ_source_up)

Γ_ext = BoundaryTriangulation(model,tags="ext")
dΓ_ext= Measure(Γ_ext,2)

GenericMeasure()

We set the laboratory ambient temperature to 25 °C.

In [21]:
temp_amb_C = 25
temp_amb_K = temp_amb_C + 273.15
g(x,t::Real) = temp_amb_K 
g(t::Real) = x -> g(x,t)
U = TransientTrialFESpace(V,g)

TransientTrialFESpace{UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{Int32}}, TrialFESpace{UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{Int32}}}}(UnconstrainedFESpace(), g, TrialFESpace())

Formulacion debil del problema

In [30]:
alpha(t) = 0.000094239 #  Proportional to the thermal diffusivity of aluminum
Ay=0.000346
Az=0.04152
Ax=0.00048
potencia_res_un_area = 110 # Power of the resistor per unit area
fuente = potencia_res_un_area/229
fuente_x= fuente *Ax
fuente_y= fuente*Ay
fuente_z= fuente*Az


0.01994410480349345

In [32]:
res(t,u,v) = ∫( ∂t(u)*v + alpha(t)*(∇(u)⋅∇(v)) )dΩ -∫(v *(fuente_z))*dΓ_souce_up-∫(v * (fuente_z) )*dΓ_souce_down -∫(v * (fuente_y) )*dΓ_souce_yL -∫(v * (fuente_y) )*dΓ_souce_y0 -∫(v * (fuente_x))*dΓ_souce_xL -∫(v * (fuente_x) )*dΓ_souce_x0
jac(t,u,du,v) = ∫( alpha(t)*(∇(du)⋅∇(v)) )dΩ
jac_t(t,u,duₜ,v) = ∫( duₜ*v )dΩ
op = TransientFEOperator(res,jac,jac_t,U,V)

TransientFEOperatorFromWeakForm()

In [23]:
linear_solver = LUSolver()


LUSolver()

The simulation is configured to run from time 
t=0 to 
t=10, using a time step of 0.05.

In [24]:
Δt = 0.05
θ = 0.5
ode_solver = ThetaMethod(linear_solver,Δt,θ)

ThetaMethod()

In [25]:
u₀ = interpolate_everywhere(temp_amb_K,U(0.0))
t₀ = 0.0
T = 10.0
uₕₜ = solve(ode_solver,op,u₀,t₀,T)


Gridap.ODEs.TransientFETools.TransientFESolution(GenericODESolution(), TransientTrialFESpace{UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{Int32}}, TrialFESpace{UnconstrainedFESpace{Vector{Float64}, Gridap.FESpaces.NodeToDofGlue{Int32}}}}(UnconstrainedFESpace(), g, TrialFESpace()))

To save the temperature distribution over time and generate a time-lapse (movie), the results are stored in a designated output directory.

In [28]:
dir = #name of the directoory "/home/dani/Dani/TAMI_2025/PRUEBA_DIRICH_INFORME_2"
mkpath(dir)

createpvd(dir) do pvd
        file_0 = dir * "/solution_0.00.vtu"
        pvd[0.0] = createvtk(Ω, file_0, cellfields=["u" =>u₀])
    for (uₕ, t) in uₕₜ
        file = dir * "/solution_$t.vtu"
        pvd[t] = createvtk(Ω, file, cellfields=["u" => uₕ])
    end
end;